<a href="https://colab.research.google.com/github/ace26597/Riiid-Knowledge-Tracing/blob/main/Riiid_lgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone --recursive https://github.com/microsoft/LightGBM
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 20577 (delta 0), reused 0 (delta 0), pack-reused 20573
Receiving objects: 100% (20577/20577), 16.02 MiB | 30.21 MiB/s, done.
Resolving deltas: 100% (15026/15026), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 28.94 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-

In [ ]:
import pandas as pd

train_pickle ='/content/drive/My Drive/Colab Data/riiid/lgb_train.p'
valid_pickle = '/content/drive/My Drive/Colab Data/riiid/lgb_valid.p'

train = pd.read_pickle(train_pickle)
valid = pd.read_pickle(valid_pickle)

FileNotFoundError: ignored

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

FEATS = ['row_id', 'user_id', 'content_id', 'content_type_id',
       'prior_question_elapsed_time',
       'prior_question_had_explanation', 'answered_correctly_sum_u', 'count_u',
       'answered_correctly_avg_u', 'answered_correctly_avg_c',
       'prior_question_elapsed_time_mean', 'question_id', 'part']
 
TARGET = ['answered_correctly']

In [ ]:
"""model = lgb.train(
                    {
                        'objective': 'binary',
                        #'tree_method': 'hist'
                        'metric': 'auc',
                        #'learning_rate': 0.08,
                        #'max_bin': 800,
                        'boosting_type': 'dart',
                        'device_type':'gpu',
                        #'num_leaves': 31,
                        #'feature_fraction': 0.9,
                        #'bagging_fraction': 0.8,
                        #'bagging_freq': 5,
                        'verbose' : 0
                    },
                    lgb_train,
                    valid_sets=[lgb_train, lgb_valid],
                    verbose_eval=50,
                    num_boost_round=5000,
                    #early_stopping_rounds=10,
                    feature_name = FEATS,
                    categorical_feature = categorical
                )

print('auc:', roc_auc_score(valid[TARGET], model.predict(valid[FEATS])))

lgb.plot_importance(model)"""

"model = lgb.train(\n                    {\n                        'objective': 'binary',\n                        #'tree_method': 'hist'\n                        'metric': 'auc',\n                        #'learning_rate': 0.08,\n                        #'max_bin': 800,\n                        'boosting_type': 'dart',\n                        'device_type':'gpu',\n                        #'num_leaves': 31,\n                        #'feature_fraction': 0.9,\n                        #'bagging_fraction': 0.8,\n                        #'bagging_freq': 5,\n                        'verbose' : 0\n                    },\n                    lgb_train,\n                    valid_sets=[lgb_train, lgb_valid],\n                    verbose_eval=50,\n                    num_boost_round=5000,\n                    #early_stopping_rounds=10,\n                    feature_name = FEATS,\n                    categorical_feature = categorical\n                )\n\nprint('auc:', roc_auc_score(valid[TARGET]

In [ ]:
def objective(trial): 
       
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 32, 512),
            'boosting_type': 'gbdt',
            'max_bin': trial.suggest_int('max_bin', 10, 200),
            'objective': 'binary',
            'metric': 'auc',
            'device_type':'gpu',
            'max_depth': trial.suggest_int('max_depth', 4, 100),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 30),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'min_child_samples': trial.suggest_int('min_child_samples', 4, 150),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
            'early_stopping_rounds': 100
            }
    
    lgb_train = lgb.Dataset(train[FEATS],label = train[TARGET],free_raw_data = False)
    lgb_valid = lgb.Dataset(valid[FEATS],label = valid[TARGET])

    categorical = [3,5,12]

    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_valid], verbose_eval=100, num_boost_round=1300)
    val_pred = model.predict(valid[FEATS])
    score = roc_auc_score(valid[TARGET], val_pred)
    print("AUC = {score}",score)
    return score

In [ ]:
!pip install --quiet optuna
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

[I 2020-11-18 22:00:57,281] A new study created in memory with name: no-name-68ec2541-2f6c-4962-8313-87506afbe0dc
[W 2020-11-18 22:00:57,288] Trial 0 failed because of the following error: TypeError("suggest_categorical() missing 1 required positional argument: 'choices'",)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/_optimize.py", line 189, in _run_trial
    value = func(trial)
  File "<ipython-input-24-bd6af472cc15>", line 5, in objective
    'boosting_type': trial.suggest_categorical(['gbdt','dart']),
TypeError: suggest_categorical() missing 1 required positional argument: 'choices'


TypeError: ignored

In [ ]:
print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 5
Best trial:
  Value: 0.762826684770459
  Params: 
    num_leaves: 229
    max_bin: 140
    max_depth: 15
    min_child_weight: 9
    feature_fraction: 0.5306317286297064
    bagging_fraction: 0.2788538066534348
    bagging_freq: 10
    min_child_samples: 85
    lambda_l1: 0.12162941430780584
    lambda_l2: 4.902535927357699e-05
